In [1]:
from featureExtraction import featureExtraction
from modelGenerator import modelGenerator
import pandas as pd
import matplotlib.pyplot as plt
from utils import reset_seeds


reset_seeds(seed_value=39)
dataID = "raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27"
modelFreqIDs = ["linearSGD","svr","mlp"]
modelTimeIDs = ["lstm","esn","tcn","seq2point","rocket"]
params = {}

dataFreq = featureExtraction(dataID,statorFreqs=[37],testsID=[21],featsDomain="freq")  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
dataFreqTest = featureExtraction(dataID,statorFreqs=[37],testsID=[24],featsDomain="freq",scaler_params=dataFreq.get_scaler_params())  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27

dataTime = featureExtraction(dataID,statorFreqs=[37],testsID=[21],featsDomain="time",timesteps=1100)  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27
dataTimeTest = featureExtraction(dataID,statorFreqs=[37],testsID=[24],featsDomain="time",timesteps=1100,scaler_params=dataTime.get_scaler_params())  # raw_data_10000_samples_fm_20000_tests_Prueba_21_Prueba_24_Prueba_27


MODELSFREQ = []
MODELSTIME = []

# Training freq-based models
for modelID in modelFreqIDs:
    model = modelGenerator(modelID=modelID, data=dataFreq,params=params)
    model.train()
    MODELSFREQ.append(model)

for modelID in modelTimeIDs:
    model = modelGenerator(modelID=modelID, data=dataTime,params=params)
    model.train()
    MODELSTIME.append(model)



2023-07-24 13:26:30.674253: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/diego/.conda/envs/hiwind/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


training <class 'models.linearSGD.linearSGD'>
Test MSE: 4.9313776788566805    Test MAE: 1.8677998424220716  
Test MSE: 1.403991811831425    Test MAE: 0.9570821180950371  


/home/diego/.conda/envs/hiwind/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
2023-07-24 13:26:53.379293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 6)]               0         
                                                                 
 dense_0 (Dense)             (None, 130)               910       
                                                                 
 dense_1 (Dense)             (None, 140)               18340     
                                                                 
 dense (Dense)               (None, 1)                 141       
                                                                 
Total params: 19,391
Trainable params: 19,391
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300


2023-07-24 13:26:53.631406: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22309 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


124/175 [====================>.........] - ETA: 0s - loss: 14708.1465 - mean_squared_error: 14708.1465 - mean_absolute_error: 117.7460

2023-07-24 13:26:54.277897: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


175/175 [==============================] - 1s 1ms/step - loss: 13622.6875 - mean_squared_error: 13622.6875 - mean_absolute_error: 112.7915 - val_loss: 9543.0479 - val_mean_squared_error: 9543.0479 - val_mean_absolute_error: 93.6938
Epoch 2/300
175/175 [==============================] - 0s 986us/step - loss: 7194.4712 - mean_squared_error: 7194.4712 - mean_absolute_error: 79.9705 - val_loss: 5026.0435 - val_mean_squared_error: 5026.0435 - val_mean_absolute_error: 66.0147
Epoch 3/300
175/175 [==============================] - 0s 981us/step - loss: 3789.1750 - mean_squared_error: 3789.1750 - mean_absolute_error: 56.7188 - val_loss: 2625.0796 - val_mean_squared_error: 2625.0796 - val_mean_absolute_error: 46.5579
Epoch 4/300
175/175 [==============================] - 0s 971us/step - loss: 2039.2650 - mean_squared_error: 2039.2650 - mean_absolute_error: 41.0430 - val_loss: 1462.6588 - val_mean_squared_error: 1462.6588 - val_mean_absolute_error: 34.6811
Epoch 5/300
175/175 [==================

2023-07-24 13:27:02.454867: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


25/25 [==============================] - 2s 35ms/step - loss: 8980.1543 - mean_squared_error: 8980.1533 - mean_absolute_error: 81.1843 - val_loss: 1117.6073 - val_mean_squared_error: 1117.6073 - val_mean_absolute_error: 22.0892
Epoch 2/300
25/25 [==============================] - 1s 25ms/step - loss: 819.1934 - mean_squared_error: 819.1934 - mean_absolute_error: 22.7763 - val_loss: 754.7025 - val_mean_squared_error: 754.7025 - val_mean_absolute_error: 22.9519
Epoch 3/300
25/25 [==============================] - 1s 27ms/step - loss: 784.9824 - mean_squared_error: 784.9824 - mean_absolute_error: 22.3251 - val_loss: 745.3697 - val_mean_squared_error: 745.3697 - val_mean_absolute_error: 22.2434
Epoch 4/300
25/25 [==============================] - 1s 27ms/step - loss: 775.8477 - mean_squared_error: 775.8477 - mean_absolute_error: 22.0825 - val_loss: 738.7480 - val_mean_squared_error: 738.7480 - val_mean_absolute_error: 22.0689
Epoch 5/300
25/25 [==============================] - 0s 20ms/ste

In [2]:
X_test_freq = dataFreqTest.X
y_test_freq = dataFreqTest.y
Y_test_est_freq = []
for model in MODELSFREQ:
    Y_test_est_freq.append(model.predict(X_test_freq))

X_test_time = dataTimeTest.X
y_test_time = dataTimeTest.y[:,-1]
Y_test_est_time = []
for model in MODELSTIME:
    Y_test_est_time.append(model.predict(X_test_time))

9/9 [==============================] - 0s 677us/step


9/9 [==============================] - 0s 8ms/step
Getting  states
Getting outputs
9/9 [==============================] - 0s 6ms/step


# Frequency-based qualitative analysis
In figure, the measured temperature (ground truth) is compared to the temperature values estimated by freq-based models. 

In [5]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
output_notebook()
colors = ["royalblue","sandybrown","seagreen"]
MODEL_LABELS = {"mlp": "MLP","svr": "SVR", "linearSGD":"MLR"}

# filtering some temperature range
min_temp = 150
idx_fig = y_test_freq>min_temp
y_test_fig = y_test_freq[idx_fig]
Y_test_est_fig = [Y_test_est_freq[y][idx_fig] for y in range(len(modelFreqIDs))]

p = figure(width=600,height=450)
p.line(np.arange(len(y_test_fig)),y_test_fig, line_width=2,line_alpha=0.8, line_color='gray', legend_label="Ground truth" )
for ii,model in enumerate(MODELSFREQ):
    p.line(np.arange(len(y_test_fig)),Y_test_est_fig[ii], line_width=1,  line_color=colors[ii],line_alpha=0.6, legend_label=MODEL_LABELS[modelFreqIDs[ii]])
p.legend.location = 'bottom_right'
p.yaxis.axis_label = r"T stator (ºC)"
p.yaxis.axis_label_text_font_size = "13pt"
p.yaxis.major_label_text_font_size = "10pt"
p.xaxis.axis_label = r"sample"
p.xaxis.axis_label_text_font_size = "13pt"
p.xaxis.major_label_text_font_size = "10pt"
p.legend.label_text_font_size = "13pt"


show(p)


Loading BokehJS ...

# Time-based qualitative analysis
In figure, the measured temperature (ground truth) is compared to the temperature values estimated by time-based models. 

In [7]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np
output_notebook()
colors = ["#9B59B6","#2980B9","#1ABC9C","#F1C40F","#2E4053"]
MODEL_LABELS_TIME = {"lstm": "LSTM","rocket": "rocket", "seq2point":"CNN","tcn": "TCN", "esn":"ESN"}

selected_models = ["lstm","rocket","esn"]

# filtering some temperature range
min_temp = 0
idx_fig = y_test_time>min_temp
y_test_fig = y_test_time[idx_fig]
Y_test_est_fig = [Y_test_est_time[modelTimeIDs.index(model)][idx_fig] for model in selected_models]

p = figure(width=600,height=450)
p.line(np.arange(len(y_test_fig)),y_test_fig, line_width=2,line_alpha=0.8, line_color='gray', legend_label="Ground truth" )
for ii,model in enumerate(selected_models):
    p.line(np.arange(len(y_test_fig)),Y_test_est_fig[ii], line_width=1,  line_color=colors[ii],line_alpha=0.6, legend_label=MODEL_LABELS_TIME[selected_models[ii]])
p.legend.location = 'bottom_right'
p.yaxis.axis_label = r"T stator (ºC)"
p.yaxis.axis_label_text_font_size = "13pt"
p.yaxis.major_label_text_font_size = "10pt"
p.xaxis.axis_label = r"sample"
p.xaxis.axis_label_text_font_size = "13pt"
p.xaxis.major_label_text_font_size = "10pt"
p.legend.label_text_font_size = "13pt"


show(p)

Loading BokehJS ...

In [20]:
feat = 0
datos_error = dataTimeTest.X[61][...,feat]
datos_normal = dataTimeTest.X[62][...,feat]

p = figure(width=600,height=450)
p.line(np.arange(len(datos_error)),datos_error, line_width=2,line_alpha=0.8, line_color='orange', legend_label="error" )
p.line(np.arange(len(datos_error)),datos_normal, line_width=2,line_alpha=0.8, line_color='gray', legend_label="normal" )
p.line(np.arange(len(datos_error)),datos_normal-datos_error, line_width=2,line_alpha=0.8, line_color='green', legend_label="normal" )
show(p)

: 